# Импорт библиотек

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install pytorch-transformers

In [ ]:
! pip install pymorphy3

In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

import torch

import string
import re

from tqdm import tqdm, trange, tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

import random
import pymorphy3

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import AdamW, BertForSequenceClassification, WarmupLinearSchedule
import io
from IPython.display import clear_output

In [ ]:
import os

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
from collections import Counter

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == 'cpu':
    print('cpu')
else:
    n_gpu = torch.cuda.device_count()
    print(torch.cuda.get_device_name(0))

Tesla T4


In [ ]:
dataset = pd.read_excel('/content/drive/MyDrive/Guilford/cleaned data Guilford.xlsx')

dataset.head(5)

,Unnamed: 0,ID,attempt,n,answer,origNew,prepr_text,simple_category
0,0,PTxGzRMvLY,1,1,сложить из спичек домик,3,сложить из спичек домик,5
1,1,PTxGzRMvLY,1,2,использовать спички как опору чего - нибудь,5,использовать спички как опору чего нибудь,6
2,2,PTxGzRMvLY,1,3,спичками мжно ковыряться в зубах,3,спичками мжно ковыряться в зубах,6
3,3,PTxGzRMvLY,1,4,"если сложить спички в виде колодца, можно испо...",5,если сложить спички в виде колодца можно испол...,6
4,4,PTxGzRMvLY,1,5,"если разукрасить спички, то можно сложить их н...",2,если разукрасить спички то можно сложить их на...,5


In [ ]:
dataset.shape

(7356, 8)

In [ ]:
#фиксируем seed
def init_random_seed(value=0):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
    torch.backends.cudnn.deterministic = True

RANDOM_SEED = 1500

init_random_seed(value=RANDOM_SEED)

# Логиты моделей - 3/4

In [ ]:
sentences_bert = ["[CLS] " + sentence + " [SEP]" for sentence in dataset['prepr_text'].values]

In [ ]:
len(sentences_bert)

7356

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# base -model | uncased - все слова в нижнем регистре
tokenized_texts = [tokenizer.tokenize(sent[:85]) for sent in sentences_bert]

In [ ]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(
    input_ids,
    maxlen=128,
    dtype="long",
    truncating="post",
    padding="post"
)
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]

In [ ]:
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)

In [ ]:
data = TensorDataset(input_ids, attention_masks)
dataloader = DataLoader(
    data,
    batch_size= 32
)

In [ ]:
def get_logits(model, dataloader):
    # Валидация
    # Переводим модель в evaluation mode
    model.eval()

    all_logits = []

    for batch in tqdm_notebook(dataloader):
        # добавляем батч для вычисления на GPU
        batch = tuple(t.to(device) for t in batch)

        # Распаковываем данные из dataloader
        b_input_ids, b_input_mask = batch

        # При использовании .no_grad() модель не будет считать и хранить градиенты.
        # Это ускорит процесс предсказания меток для тестовых данных.
        with torch.no_grad():
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        # Перемещаем logits на CPU для дальнейшей работы
        logits = logits[0].detach().cpu().numpy()

        all_logits.extend(logits)

    return all_logits

# RuntimeError: Expected all tensors to be on the same device,
# but found at least two devices, cuda:0 and cpu! (when checking
# argument for argument index in method wrapper_CUDA__index_select)

# OutOfMemoryError: CUDA out of memory. Tried to allocate 3.33 GiB.
# GPU 0 has a total capacity of 14.75 GiB of which 967.06 MiB is free.
# Process 2078 has 13.80 GiB memory in use. Of the allocated memory 13.40 GiB
# is allocated by PyTorch, and 297.12 MiB is reserved by PyTorch but
# unallocated. If reserved but unallocated memory is large try setting
# PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.
# See documentation for Memory Management (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from pytorch_transformers import AdamW, BertForSequenceClassification

**1 модель** - модель, предсказывающая оригинальность (3)

BRT_orig_3_0.008

In [ ]:
model1 = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

In [ ]:
model1 = model1.to(device)

In [ ]:
model1.load_state_dict(torch.load('/content/drive/MyDrive/Guilford/BRT_orig_3_0.008.ckpt'))

<All keys matched successfully>

In [ ]:
model1_logits = get_logits(model1, dataloader)

  0%|          | 0/230 [00:00<?, ?it/s]

In [ ]:
len(model1_logits)

7356

In [ ]:
dataset = dataset.join(pd.DataFrame(model1_logits,
                        columns=['orig_v1', 'orig_v2','orig_v3']), how='left')

In [ ]:
dataset

,Unnamed: 0,ID,attempt,n,answer,origNew,prepr_text,simple_category,orig_v1,orig_v2,orig_v3
0,0,PTxGzRMvLY,1,1,сложить из спичек домик,3,сложить из спичек домик,5,3.681304,-2.685261,-2.385877
1,1,PTxGzRMvLY,1,2,использовать спички как опору чего - нибудь,5,использовать спички как опору чего нибудь,6,-2.919616,-1.976754,5.205158
2,2,PTxGzRMvLY,1,3,спичками мжно ковыряться в зубах,3,спичками мжно ковыряться в зубах,6,4.041395,-2.963273,-2.460635
3,3,PTxGzRMvLY,1,4,"если сложить спички в виде колодца, можно испо...",5,если сложить спички в виде колодца можно испол...,6,-3.026895,-1.863899,5.181743
4,4,PTxGzRMvLY,1,5,"если разукрасить спички, то можно сложить их н...",2,если разукрасить спички то можно сложить их на...,5,3.958443,-2.817475,-2.532558
...,...,...,...,...,...,...,...,...,...,...,...
7351,9256,2008_2009_320,0,2,для поджога мусора,4,для поджога мусора,1,-1.947971,3.559194,-1.452860
7352,9257,2008_2009_320,0,3,для играния,3,для играния,7,3.613889,-2.653998,-2.236441
7353,9258,2008_2009_321,0,1,зубочистка - зубы чистить,3,зубочистка зубы чистить,6,4.115974,-2.693043,-2.804608
7354,9259,2008_2009_321,0,2,как палочка - для игр,3,как палочка для игр,7,3.567055,-2.552118,-2.322904


**2 модель** - модель, предсказывающая категорию (4)

BRT_cat_4_0.049

In [ ]:
model2 = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

In [ ]:
model2 = model2.to(device)

In [ ]:
model2.load_state_dict(torch.load('/content/drive/MyDrive/Guilford/BRT_cat_4_0.049.ckpt'))

<All keys matched successfully>

In [ ]:
model2_logits = get_logits(model2, dataloader)

  0%|          | 0/230 [00:00<?, ?it/s]

In [ ]:
dataset = dataset.join(pd.DataFrame(model2_logits,
                        columns=['cat_v1', 'cat_v2','cat_v3', 'cat_v4']), how='left')

In [ ]:
dataset

,Unnamed: 0,ID,attempt,n,answer,origNew,prepr_text,simple_category,orig_v1,orig_v2,orig_v3,cat_v1,cat_v2,cat_v3,cat_v4
0,0,PTxGzRMvLY,1,1,сложить из спичек домик,3,сложить из спичек домик,5,3.681304,-2.685261,-2.385877,-1.189256,4.626387,-1.855205,-1.131889
1,1,PTxGzRMvLY,1,2,использовать спички как опору чего - нибудь,5,использовать спички как опору чего нибудь,6,-2.919616,-1.976754,5.205158,-1.326608,-1.099055,3.938310,-1.653843
2,2,PTxGzRMvLY,1,3,спичками мжно ковыряться в зубах,3,спичками мжно ковыряться в зубах,6,4.041395,-2.963273,-2.460635,-1.020064,-1.630299,4.506532,-1.679100
3,3,PTxGzRMvLY,1,4,"если сложить спички в виде колодца, можно испо...",5,если сложить спички в виде колодца можно испол...,6,-3.026895,-1.863899,5.181743,-2.341077,0.076745,3.568159,-1.647687
4,4,PTxGzRMvLY,1,5,"если разукрасить спички, то можно сложить их н...",2,если разукрасить спички то можно сложить их на...,5,3.958443,-2.817475,-2.532558,-1.583953,4.732340,-1.147014,-1.611131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7351,9256,2008_2009_320,0,2,для поджога мусора,4,для поджога мусора,1,-1.947971,3.559194,-1.452860,3.969714,-1.453619,-1.941337,-0.601652
7352,9257,2008_2009_320,0,3,для играния,3,для играния,7,3.613889,-2.653998,-2.236441,-1.839757,-1.548084,-1.201597,4.129030
7353,9258,2008_2009_321,0,1,зубочистка - зубы чистить,3,зубочистка зубы чистить,6,4.115974,-2.693043,-2.804608,-1.071780,-1.381713,4.593972,-1.938590
7354,9259,2008_2009_321,0,2,как палочка - для игр,3,как палочка для игр,7,3.567055,-2.552118,-2.322904,-2.305859,-1.592294,-0.664346,4.018522


# Переопределение оригинальности и категорий

In [ ]:
# ДЛЯ ОРИГИНАЛЬНОСТИ

def make_3_categories():
  classes = []
  for value in dataset.origNew:
    if value==1 or value ==2 or value == 3: # объединяем 1, 2 и 3 в один класс
      cls = int(0)
    elif value==4: # просто переопределяем
      cls = int(1)
    else: # объединяем 5 и 6 в один класс
      cls = int(2)
    classes.append(cls)

  dataset.origNew = classes

In [ ]:
# !!! 3 КЛАССА
make_3_categories()

In [ ]:
# ДЛЯ КАТЕГОРИЙ

def drop_small_categories():
  classes = []
  for value in dataset.simple_category:
    if value in [9, 8, 2, 3, 4]:
      cls = 98234
    elif value < 2:
      cls = int(value-1)
    else:
      cls = int(value-4)
    classes.append(cls)

  dataset.simple_category = classes

In [ ]:
# !!! 4 КЛАССА
drop_small_categories()
dataset = dataset.drop(dataset[(dataset.simple_category == 98234)].index)

In [ ]:
dataset

,Unnamed: 0,ID,attempt,n,answer,origNew,prepr_text,simple_category,orig_v1,orig_v2,orig_v3,cat_v1,cat_v2,cat_v3,cat_v4
0,0,PTxGzRMvLY,1,1,сложить из спичек домик,0,сложить из спичек домик,1,3.681304,-2.685261,-2.385877,-1.189256,4.626387,-1.855205,-1.131889
1,1,PTxGzRMvLY,1,2,использовать спички как опору чего - нибудь,2,использовать спички как опору чего нибудь,2,-2.919616,-1.976754,5.205158,-1.326608,-1.099055,3.938310,-1.653843
2,2,PTxGzRMvLY,1,3,спичками мжно ковыряться в зубах,0,спичками мжно ковыряться в зубах,2,4.041395,-2.963273,-2.460635,-1.020064,-1.630299,4.506532,-1.679100
3,3,PTxGzRMvLY,1,4,"если сложить спички в виде колодца, можно испо...",2,если сложить спички в виде колодца можно испол...,2,-3.026895,-1.863899,5.181743,-2.341077,0.076745,3.568159,-1.647687
4,4,PTxGzRMvLY,1,5,"если разукрасить спички, то можно сложить их н...",0,если разукрасить спички то можно сложить их на...,1,3.958443,-2.817475,-2.532558,-1.583953,4.732340,-1.147014,-1.611131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7351,9256,2008_2009_320,0,2,для поджога мусора,1,для поджога мусора,0,-1.947971,3.559194,-1.452860,3.969714,-1.453619,-1.941337,-0.601652
7352,9257,2008_2009_320,0,3,для играния,0,для играния,3,3.613889,-2.653998,-2.236441,-1.839757,-1.548084,-1.201597,4.129030
7353,9258,2008_2009_321,0,1,зубочистка - зубы чистить,0,зубочистка зубы чистить,2,4.115974,-2.693043,-2.804608,-1.071780,-1.381713,4.593972,-1.938590
7354,9259,2008_2009_321,0,2,как палочка - для игр,0,как палочка для игр,3,3.567055,-2.552118,-2.322904,-2.305859,-1.592294,-0.664346,4.018522


In [ ]:
dataset.shape

(6855, 15)

In [ ]:
!pip install openpyxl

In [ ]:
dataset.to_excel(f'/content/drive/MyDrive/Guilford/logits 34 Guilford.xlsx')

# Логиты моделей - 6/9

In [ ]:
dataset = pd.read_excel('/content/drive/MyDrive/Guilford/cleaned data Guilford.xlsx')
dataset.head(5)

,Unnamed: 0,ID,attempt,n,answer,origNew,prepr_text,simple_category
0,0,PTxGzRMvLY,1,1,сложить из спичек домик,3,сложить из спичек домик,5
1,1,PTxGzRMvLY,1,2,использовать спички как опору чего - нибудь,5,использовать спички как опору чего нибудь,6
2,2,PTxGzRMvLY,1,3,спичками мжно ковыряться в зубах,3,спичками мжно ковыряться в зубах,6
3,3,PTxGzRMvLY,1,4,"если сложить спички в виде колодца, можно испо...",5,если сложить спички в виде колодца можно испол...,6
4,4,PTxGzRMvLY,1,5,"если разукрасить спички, то можно сложить их н...",2,если разукрасить спички то можно сложить их на...,5


In [ ]:
sentences_bert = ["[CLS] " + sentence + " [SEP]" for sentence in dataset['prepr_text'].values]

In [ ]:
len(sentences_bert)

7356

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# base -model | uncased - все слова в нижнем регистре
tokenized_texts = [tokenizer.tokenize(sent[:85]) for sent in sentences_bert]

In [ ]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(
    input_ids,
    maxlen=128,
    dtype="long",
    truncating="post",
    padding="post"
)
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]

In [ ]:
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)

In [ ]:
data = TensorDataset(input_ids, attention_masks)
dataloader = DataLoader(
    data,
    batch_size= 32
)

In [ ]:
def get_logits(model, dataloader):
    # Валидация
    # Переводим модель в evaluation mode
    model.eval()

    all_logits = []

    for batch in tqdm_notebook(dataloader):
        # добавляем батч для вычисления на GPU
        batch = tuple(t.to(device) for t in batch)

        # Распаковываем данные из dataloader
        b_input_ids, b_input_mask = batch

        # При использовании .no_grad() модель не будет считать и хранить градиенты.
        # Это ускорит процесс предсказания меток для тестовых данных.
        with torch.no_grad():
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        # Перемещаем logits на CPU для дальнейшей работы
        logits = logits[0].detach().cpu().numpy()

        all_logits.extend(logits)

    return all_logits

# RuntimeError: Expected all tensors to be on the same device,
# but found at least two devices, cuda:0 and cpu! (when checking
# argument for argument index in method wrapper_CUDA__index_select)

# OutOfMemoryError: CUDA out of memory. Tried to allocate 3.33 GiB.
# GPU 0 has a total capacity of 14.75 GiB of which 967.06 MiB is free.
# Process 2078 has 13.80 GiB memory in use. Of the allocated memory 13.40 GiB
# is allocated by PyTorch, and 297.12 MiB is reserved by PyTorch but
# unallocated. If reserved but unallocated memory is large try setting
# PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.
# See documentation for Memory Management (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from pytorch_transformers import AdamW, BertForSequenceClassification

**3 модель** - модель, предсказывающая оригинальность (6)

BRT_orig_6_0.236

In [ ]:
model3 = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)

In [ ]:
model3 = model3.to(device)

In [ ]:
model3.load_state_dict(torch.load('/content/drive/MyDrive/Guilford/BRT_orig_6_0.236.ckpt'))

<All keys matched successfully>

In [ ]:
model3_logits = get_logits(model3, dataloader)

  0%|          | 0/230 [00:00<?, ?it/s]

In [ ]:
len(model3_logits)

7356

In [ ]:
dataset = dataset.join(pd.DataFrame(model3_logits,
                        columns=['orig_v1', 'orig_v2','orig_v3',
                                 'orig_v4', 'orig_v5', 'orig_v6']), how='left')

In [ ]:
dataset

,Unnamed: 0,ID,attempt,n,answer,origNew,prepr_text,simple_category,orig_v1,orig_v2,orig_v3,orig_v4,orig_v5,orig_v6
0,0,PTxGzRMvLY,1,1,сложить из спичек домик,3,сложить из спичек домик,5,-1.050824,-0.965074,4.772342,-1.234295,-0.542633,-1.480853
1,1,PTxGzRMvLY,1,2,использовать спички как опору чего - нибудь,5,использовать спички как опору чего нибудь,6,-1.788807,-0.206116,-1.528240,0.819028,3.540708,-0.607792
2,2,PTxGzRMvLY,1,3,спичками мжно ковыряться в зубах,3,спичками мжно ковыряться в зубах,6,-1.544888,-1.201231,4.198406,-0.917910,0.504700,-1.538587
3,3,PTxGzRMvLY,1,4,"если сложить спички в виде колодца, можно испо...",5,если сложить спички в виде колодца можно испол...,6,-1.670379,-0.707951,-1.018482,-0.240077,4.071992,-0.352007
4,4,PTxGzRMvLY,1,5,"если разукрасить спички, то можно сложить их н...",2,если разукрасить спички то можно сложить их на...,5,-1.380270,1.989305,-1.418480,-0.209274,2.677757,-0.830254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7351,9256,2008_2009_320,0,2,для поджога мусора,4,для поджога мусора,1,-2.730381,-1.149435,0.242161,4.033585,0.797759,-1.752713
7352,9257,2008_2009_320,0,3,для играния,3,для играния,7,-1.902111,-1.503589,4.069835,-0.093787,0.595744,-1.772283
7353,9258,2008_2009_321,0,1,зубочистка - зубы чистить,3,зубочистка зубы чистить,6,-0.894824,-0.907597,4.772493,-1.241397,-1.038014,-1.297550
7354,9259,2008_2009_321,0,2,как палочка - для игр,3,как палочка для игр,7,-1.820840,-1.049067,2.967299,-0.375491,1.514886,-1.421187


**4 модель** - модель, предсказывающая категорию (9)

BRT_cat_9_0.002

In [ ]:
model4 = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=9)

In [ ]:
model4 = model4.to(device)

In [ ]:
model4.load_state_dict(torch.load('/content/drive/MyDrive/Guilford/BRT_cat_9_0.002.ckpt'))

<All keys matched successfully>

In [ ]:
model4_logits = get_logits(model4, dataloader)

  0%|          | 0/230 [00:00<?, ?it/s]

In [ ]:
dataset = dataset.join(pd.DataFrame(model4_logits,
                        columns=['cat_v1', 'cat_v2','cat_v3', 'cat_v4',
                                 'cat_v5', 'cat_v6', 'cat_v7', 'cat_v8', 'cat_v9']), how='left')

In [ ]:
dataset

,Unnamed: 0,ID,attempt,n,answer,origNew,prepr_text,simple_category,orig_v1,orig_v2,...,orig_v6,cat_v1,cat_v2,cat_v3,cat_v4,cat_v5,cat_v6,cat_v7,cat_v8,cat_v9
0,0,PTxGzRMvLY,1,1,сложить из спичек домик,3,сложить из спичек домик,5,-1.050824,-0.965074,...,-1.480853,-0.977374,-2.093006,-2.025653,-1.560774,7.735858,-0.845992,-0.213102,-1.882106,-2.248635
1,1,PTxGzRMvLY,1,2,использовать спички как опору чего - нибудь,5,использовать спички как опору чего нибудь,6,-1.788807,-0.206116,...,-0.607792,-0.647404,-0.256699,-1.394448,-2.822276,-1.453254,6.590232,0.225316,-2.341004,-1.730908
2,2,PTxGzRMvLY,1,3,спичками мжно ковыряться в зубах,3,спичками мжно ковыряться в зубах,6,-1.544888,-1.201231,...,-1.538587,-1.988488,-0.471610,-1.662081,-2.336824,0.137961,8.230809,-1.371563,-2.219541,-2.200917
3,3,PTxGzRMvLY,1,4,"если сложить спички в виде колодца, можно испо...",5,если сложить спички в виде колодца можно испол...,6,-1.670379,-0.707951,...,-0.352007,-2.750197,-1.154556,1.661113,-2.023335,-0.358008,6.483499,-0.648363,-1.122851,-1.342096
4,4,PTxGzRMvLY,1,5,"если разукрасить спички, то можно сложить их н...",2,если разукрасить спички то можно сложить их на...,5,-1.380270,1.989305,...,-0.830254,-1.444094,-1.559943,-2.302702,-1.675137,7.446181,0.325887,-0.207006,-2.329860,-2.618991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7351,9256,2008_2009_320,0,2,для поджога мусора,4,для поджога мусора,1,-2.730381,-1.149435,...,-1.752713,7.091044,-1.102022,-1.391652,-1.487136,-2.373131,-1.815341,0.802976,-1.485990,-0.576146
7352,9257,2008_2009_320,0,3,для играния,3,для играния,7,-1.902111,-1.503589,...,-1.772283,0.478407,-1.807752,-1.432956,-1.523789,-2.321021,-0.738640,7.275775,-1.597619,-0.570007
7353,9258,2008_2009_321,0,1,зубочистка - зубы чистить,3,зубочистка зубы чистить,6,-0.894824,-0.907597,...,-1.297550,-2.033352,-0.476836,-1.119638,-2.192447,-0.184970,8.271646,-1.586977,-1.924915,-2.102520
7354,9259,2008_2009_321,0,2,как палочка - для игр,3,как палочка для игр,7,-1.820840,-1.049067,...,-1.421187,0.316067,-1.977534,-1.299032,-1.289200,-2.286223,-0.501457,7.127117,-1.480587,-0.573279


In [ ]:
dataset.to_excel(f'/content/drive/MyDrive/Guilford/logits 69 Guilford.xlsx')